## Here's a notebook of how I cleaned the data I used in my example Bokeh app, VirtualDive
**NOTE: This notebook is optional: I've already provided cleaned data files in the data folder of this repo so that this does not need to be run before the Bokeh notebook**

1) Let's import our packages! For this bit all I used was Pandas and Numpy:

In [1]:
import pandas as pd
import numpy as np

2) Let's load our data into a couple Pandas dataframes! ReefLocations is actually from http://reefbase.org/ and the SEDAC 2000 population density data is actually from https://neo.sci.gsfc.nasa.gov/view.php?datasetId=SEDAC_POP . I've just reuploaded both to GitHub to make sure the files don't move or change in the future.

In [22]:
reeflocations=pd.read_csv("https://raw.githubusercontent.com/mistergroot/VirtualDive/master/data/ReefLocations.csv")
pop = pd.read_csv("https://raw.githubusercontent.com/mistergroot/VirtualDive/master/data/SEDAC_POP_2000-01-01_rgb_1440x720.SS.CSV")

3) Let's see what we have to work with:

In [23]:
reeflocations.head(4)

,ID,REGION,SUBREGION,COUNTRY,LOCATION,LAT,LON,REEF_SYSTEM,REEF_TYPE,REEF_NAME,WATER_DEPTH,ISLAND_NAME,PROTECTED,TOURISM,COUNTRY_CODE,SIZE
0,62,Pacific,Southwest Pacific,Fiji,NaN,-16.00000,-179.98333,Vanua Levu,Fringing,Cikobia,NaN,Vanua Levu,0.0,0,FJI,3
1,4475,Pacific,Southwest Pacific,Fiji,NaN,-17.50000,-179.95000,Vanua Balavu,Barrier,Daku Barrier Reef,NaN,NaN,0.0,0,FJI,3
2,4457,Pacific,Southwest Pacific,Fiji,NaN,-16.66667,-179.83333,Taveuni,Fringing,Korolevu,NaN,NaN,0.0,0,FJI,3
3,4459,Pacific,Southwest Pacific,Fiji,NaN,-16.73333,-179.83333,Taveuni,Fringing,Viubani,NaN,NaN,0.0,0,FJI,3


In [24]:
pop.head(4)

,lat/lon,-179.875,-179.625,-179.375,-179.125,-178.875,-178.625,-178.375,-178.125,-177.875,...,177.625,177.875,178.125,178.375,178.625,178.875,179.125,179.375,179.625,179.875
0,89.875,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,...,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0
1,89.625,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,...,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0
2,89.375,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,...,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0
3,89.125,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,...,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0


4) Let's start with the pop data. It looks pretty ugly. It's a gridded dataset where the top row and left column are longitudes and latitudes, respectively. I don't like that the latitudes are in an unindexed column. Let's change that. It's also going to make the index look weird with the name 'lat/lon' there, so let's get rid of it at the same time:

In [25]:
pop = pop.set_index('lat/lon')
del pop.index.name

In [26]:
pop.head(2)

,-179.875,-179.625,-179.375,-179.125,-178.875,-178.625,-178.375,-178.125,-177.875,-177.625,...,177.625,177.875,178.125,178.375,178.625,178.875,179.125,179.375,179.625,179.875
89.875,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,...,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0
89.625,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,...,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0


5) Okay now let's go ahead and convert this gridded data to "lat", "lon", and "popdens" columns. This line of code essentially finds all `'item'`s in the dataframe and displays its column and row name side by side. I wanted data greater than 250 people per km^2 (because if we start at 0 we'll have maybe 100,000 datapoints and that's excessive for my needs) and less than 99999 (because 99999 is a filler value which is equivalent to no data/no people):

In [27]:
data = pop.stack().to_frame('item').query('250 < item < 99999')

In [28]:
data.head(4)

,,item
69.375,88.125,2102.98
65.125,57.375,790.02
64.875,-147.875,382.54
64.375,40.875,819.19


6) Looks good! Formatting is a bit messed up though, so let's reset the indexed column and rename the columns to the correct thing.

In [29]:
data = data.reset_index(drop=False)
data.rename(columns={'level_0':'lat','level_1':'lon','item':'popdens'}, inplace=True)
data.head(4)

,lat,lon,popdens
0,69.375,88.125,2102.98
1,65.125,57.375,790.02
2,64.875,-147.875,382.54
3,64.375,40.875,819.19


7) We're done with the population data! Let's export it to a CSV:

In [30]:
data.to_csv("../data/popdata.csv")

8) Now let's start on the reef data. This should be easier to do since it's already in columns rather than gridded. Firstly let's check what datatype our columns are in:

In [31]:
reeflocations.dtypes

ID                int64
REGION           object
SUBREGION        object
COUNTRY          object
LOCATION         object
LAT             float64
LON             float64
REEF_SYSTEM      object
REEF_TYPE        object
REEF_NAME        object
WATER_DEPTH      object
ISLAND_NAME      object
PROTECTED       float64
TOURISM           int64
COUNTRY_CODE     object
SIZE              int64
dtype: object

9) I don't like that some numeric data is stored as objects (e.g. WATER_DEPTH) so we'll convert objects that are numeric into floats. We're not actually going to use the water depth later, I just want to do this anyway:

In [36]:
reeflocations = reeflocations.convert_objects(convert_numeric=True)
reeflocations.dtypes

/home/nickpichome/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


ID                int64
REGION           object
SUBREGION        object
COUNTRY          object
LOCATION        float64
LAT             float64
LON             float64
REEF_SYSTEM      object
REEF_TYPE        object
REEF_NAME        object
WATER_DEPTH     float64
ISLAND_NAME      object
PROTECTED       float64
TOURISM           int64
COUNTRY_CODE    float64
SIZE              int64
dtype: object

10) Now, in the unformatted data, unprotected reefs are represented with 0 and protected reefs are represented with a 1. If data is absent, I'd like to assume that they're not protected, so here we'll use `fillna` to add 0s to the missing portions.

In [37]:
reeflocations['PROTECTED'] = reeflocations['PROTECTED'].fillna(0)

11) I want to replace the 1s and 0s with "Yes" and "No" now simply because that's how I want it displayed in the glyphs later. I had trouble doing this with the Boolean values of 1 and 0 for some reason so I just convert it to an int first.

In [38]:
reeflocations['PROTECTED'] = reeflocations['PROTECTED'].astype(np.int64)
reeflocations["PROTECTED"] = reeflocations["PROTECTED"].replace(1, "Yes")
reeflocations["PROTECTED"] = reeflocations["PROTECTED"].replace(0, "No")
reeflocations.head(4)

,ID,REGION,SUBREGION,COUNTRY,LOCATION,LAT,LON,REEF_SYSTEM,REEF_TYPE,REEF_NAME,WATER_DEPTH,ISLAND_NAME,PROTECTED,TOURISM,COUNTRY_CODE,SIZE
0,62,Pacific,Southwest Pacific,Fiji,NaN,-16.00000,-179.98333,Vanua Levu,Fringing,Cikobia,NaN,Vanua Levu,No,0,NaN,3
1,4475,Pacific,Southwest Pacific,Fiji,NaN,-17.50000,-179.95000,Vanua Balavu,Barrier,Daku Barrier Reef,NaN,NaN,No,0,NaN,3
2,4457,Pacific,Southwest Pacific,Fiji,NaN,-16.66667,-179.83333,Taveuni,Fringing,Korolevu,NaN,NaN,No,0,NaN,3
3,4459,Pacific,Southwest Pacific,Fiji,NaN,-16.73333,-179.83333,Taveuni,Fringing,Viubani,NaN,NaN,No,0,NaN,3


12) Looks good! Let's split up the data into two separate dataframes based on whether or not they're protected:

In [39]:
protected = reeflocations[reeflocations["PROTECTED"] == "Yes"]
unprotected = reeflocations[reeflocations["PROTECTED"] == "No"]

13) And finally, we'll export the split and unsplit reef data to CSVs using Pandas:

In [40]:
reeflocations.to_csv("../data/reefloc.csv")
protected.to_csv("../data/protected.csv")
unprotected.to_csv("../data/unprotected.csv")

## And we're done processing the raw data! Now to move on to the actual Bokeh app